# Dependencies

In [3]:
# !pip install -r requirements.txt

In [4]:
# !pip freeze > requirements.txt

In [5]:
!pip freeze

absl-py==1.4.0
anyio==3.6.2
appnope==0.1.3
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
astunparse==1.6.3
attrs==22.2.0
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
cachetools==5.3.0
certifi==2022.12.7
cffi==1.15.1
charset-normalizer==3.1.0
click==8.1.3
comm==0.1.2
contourpy==1.0.7
cycler==0.11.0
debugpy==1.6.6
decorator==5.1.1
defusedxml==0.7.1
executing==1.2.0
fastjsonschema==2.16.3
Flask==2.2.3
flatbuffers==1.12
fonttools==4.39.0
fqdn==1.5.1
gast==0.4.0
google-auth==2.16.2
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.51.3
h5py==3.8.0
idna==3.4
importlib-metadata==6.0.0
importlib-resources==5.12.0
ipykernel==6.21.3
ipython==8.10.0
ipython-genutils==0.2.0
ipywidgets==8.0.4
isoduration==20.11.0
itsdangerous==2.1.2
jedi==0.18.2
Jinja2==3.1.2
joblib==1.2.0
jsonpointer==2.3
jsonschema==4.17.3
jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.6.3
jupyter_client==8.0.3
jupyter_core==5.2.0
jupyter_server==2.4.0
jupyter_server_termi

# PreProcessing

## Dependencies

In [6]:
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


## Convert data into numpy array

In [7]:
# import pathlib


test_mask = [] 
test_no_mask = []
# I would do this in a fancy os.loop way but I was using google colab and didn't really want to mess with the function too much
train_data_mask_dir="New_Masks_Dataset/Train/Mask"
train_data_no_mask_dir="New_Masks_Dataset/Train/Non_Mask"
val_data_mask_dir = 'New_Masks_Dataset/Validation/Mask'
val_data_no_mask_dir = 'New_Masks_Dataset/Validation/Non_Mask'
test_data_mask_dir="New_Masks_Dataset/Test/Mask"
test_data_no_mask_dir="New_Masks_Dataset/Test/Non_Mask"


In [8]:
# print(train_imagePaths_mask)

In [9]:
# https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
# https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
import cv2
# from google.colab.patches import cv2_imshow
import os

IMAGE_HEIGHT  = 100
IMAGE_WIDTH = 100
NUM_CLASSES = 2


input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT)


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        # cv2_imshow(img)
        # Has to be like this because of Python Ambiguity
        if img is not None:
            grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(grayscale, input_shape)
            resized = np.reshape(resized, input_shape[0] * input_shape[1])
            images.append(resized)
    return images


In [10]:
# Train
train_mask = load_images_from_folder(train_data_mask_dir)
train_no_mask = load_images_from_folder(train_data_no_mask_dir)
# Val
val_mask = load_images_from_folder(val_data_mask_dir)
val_no_mask = load_images_from_folder(val_data_no_mask_dir)
# Test
test_mask = load_images_from_folder(test_data_mask_dir)
test_no_mask = (load_images_from_folder(test_data_no_mask_dir))

In [14]:
y_train_mask = [1] * len(train_mask)
y_train_no_mask = [0] * len(train_no_mask)

y_val_mask = [1] * len(val_mask)
y_val_no_mask = [0] * len(val_no_mask)

y_test_mask = [1] * len(test_mask)
y_test_no_mask = [0] * len(test_no_mask)


In [15]:
df_train = pd.DataFrame({'label': y_train_mask,'mask_data': train_mask})

In [16]:
df_train = df_train.append(pd.DataFrame({'label': y_train_no_mask, 'mask_data': train_no_mask}), ignore_index=True)

/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_23789/1233198709.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(pd.DataFrame({'label': y_train_no_mask, 'mask_data': train_no_mask}), ignore_index=True)


In [17]:
df_train = df_train.sample(frac = 1)
df_train

,label,mask_data
47,1,"[96, 103, 111, 99, 85, 92, 109, 114, 114, 98, ..."
13,1,"[153, 153, 154, 149, 148, 140, 122, 111, 107, ..."
30,1,"[176, 168, 195, 152, 151, 146, 146, 132, 119, ..."
158,1,"[255, 255, 255, 254, 254, 254, 255, 255, 254, ..."
263,1,"[135, 241, 244, 245, 241, 156, 154, 153, 153, ..."
...,...,...
300,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
442,0,"[250, 254, 254, 254, 253, 253, 254, 252, 253, ..."
393,0,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ..."
514,0,"[138, 136, 152, 135, 138, 161, 148, 141, 130, ..."


In [18]:
X_train = df_train.iloc[:, 1:] / 255.0
X_train = pd.DataFrame(X_train['mask_data'].tolist())
y_train = df_train.iloc[:, :1]

# .astype('float32')

In [19]:
df_val = pd.DataFrame({'label': y_val_mask,'mask_data': val_mask})
df_val = df_val.append(pd.DataFrame({'label': y_val_no_mask, 'mask_data': val_no_mask}), ignore_index=True)

df_val = df_val.sample(frac = 1)
df_val

/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_23789/3146285510.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_val = df_val.append(pd.DataFrame({'label': y_val_no_mask, 'mask_data': val_no_mask}), ignore_index=True)


,label,mask_data
121,1,"[124, 121, 120, 123, 121, 161, 131, 131, 154, ..."
51,1,"[136, 142, 176, 176, 81, 41, 43, 46, 47, 47, 4..."
202,0,"[181, 199, 223, 216, 219, 219, 218, 215, 196, ..."
160,0,"[180, 180, 180, 181, 180, 181, 181, 181, 181, ..."
280,0,"[153, 156, 156, 155, 155, 157, 152, 155, 155, ..."
...,...,...
250,0,"[129, 126, 122, 110, 95, 93, 90, 123, 114, 125..."
183,0,"[184, 194, 183, 178, 189, 186, 182, 193, 189, ..."
115,1,"[49, 49, 49, 49, 49, 49, 49, 50, 50, 42, 27, 3..."
171,0,"[29, 33, 39, 47, 45, 42, 75, 100, 104, 106, 10..."


In [20]:
X_val = df_val.iloc[:, 1:] / 255.0
X_val = pd.DataFrame(X_val['mask_data'].tolist())
y_val = df_val.iloc[:,:1]


In [21]:
df_test = pd.DataFrame({'label': y_test_mask,'mask_data': test_mask})
df_test = df_test.append(pd.DataFrame({'label': y_test_no_mask, 'mask_data': test_no_mask}), ignore_index=True)

df_test = df_test.sample(frac = 1)
df_test


/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_23789/4284791088.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(pd.DataFrame({'label': y_test_no_mask, 'mask_data': test_no_mask}), ignore_index=True)


,label,mask_data
5,1,"[78, 64, 66, 63, 59, 54, 76, 86, 88, 89, 92, 9..."
45,1,"[230, 230, 230, 231, 231, 231, 232, 230, 232, ..."
70,0,"[91, 91, 89, 96, 135, 128, 114, 119, 102, 97, ..."
8,1,"[57, 58, 61, 65, 67, 69, 69, 70, 68, 68, 68, 6..."
69,0,"[85, 69, 74, 68, 46, 38, 48, 21, 19, 19, 19, 1..."
...,...,...
32,1,"[155, 173, 116, 165, 169, 164, 163, 154, 215, ..."
43,1,"[113, 115, 120, 121, 126, 128, 133, 135, 138, ..."
90,0,"[219, 219, 219, 219, 219, 218, 218, 217, 218, ..."
62,0,"[233, 230, 231, 232, 233, 227, 227, 233, 228, ..."


In [22]:
X_test = df_test.iloc[:, 1:] / 255.0
X_test = pd.DataFrame(X_test['mask_data'].tolist())
y_test = df_test.iloc[:, :1]


In [23]:
x_train = X_train.values.reshape(-1,100,100)
x_val = X_val.values.reshape(-1,100,100)
x_test = X_test.values.reshape(-1,100,100)

In [24]:
batch_size = 128
epochs = 15
# input image dimensions
input_shape = (100, 100, 1)


y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_val = keras.utils.to_categorical(y_val, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)



In [25]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(599, 100, 100)
(302, 100, 100)
(99, 100, 100)


# Running Model

## KNN (K_Nearest_Neighbor Classifier)

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
k_range = list(range(1, 30))
param_grid = {'n_neighbors': k_range, 
              'weights': ['uniform', 'distance']}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, verbose = 1)
grid_search.fit(X_train, y_train)
y_pred = grid_search.predict(X_test)
metrics.f1_score(y_test, y_pred, average='weighted')
print("Best Parameters:", grid_search.best_params_)
print("Accuracy Score:", grid_search.best_score_)

Fitting 5 folds for each of 58 candidates, totalling 290 fits
Best Parameters: {'n_neighbors': 27, 'weights': 'distance'}
Accuracy Score: 0.7612885154061624


## CNN (Convolutional neural network)

In [27]:

# The parameters I chose here were also from a project I did earlier, I'll link it here
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 33856)             0         
                                                                 
 dropout (Dropout)           (None, 33856)             0

2023-03-19 08:50:56.868460: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Trainable params: 86,530
Non-trainable params: 0
_________________________________________________________________


In [28]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
             EarlyStopping(monitor='val_loss', patience=100)]


history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val,y_val))

Epoch 1/15
5/5 [==============================] - 9s 2s/step - loss: 0.8087 - accuracy: 0.5058 - val_loss: 0.7663 - val_accuracy: 0.5066
Epoch 2/15
5/5 [==============================] - 7s 1s/step - loss: 0.6818 - accuracy: 0.5476 - val_loss: 0.6564 - val_accuracy: 0.5331
Epoch 3/15
5/5 [==============================] - 6s 1s/step - loss: 0.5968 - accuracy: 0.7663 - val_loss: 0.6016 - val_accuracy: 0.7682
Epoch 4/15
5/5 [==============================] - 6s 1s/step - loss: 0.5046 - accuracy: 0.8164 - val_loss: 0.4921 - val_accuracy: 0.8245
Epoch 5/15
5/5 [==============================] - 6s 1s/step - loss: 0.4170 - accuracy: 0.8197 - val_loss: 0.4241 - val_accuracy: 0.8146
Epoch 6/15
5/5 [==============================] - 7s 1s/step - loss: 0.3715 - accuracy: 0.8280 - val_loss: 0.3945 - val_accuracy: 0.8543
Epoch 7/15
5/5 [==============================] - 7s 1s/step - loss: 0.3565 - accuracy: 0.8381 - val_loss: 0.3798 - val_accuracy: 0.8642
Epoch 8/15
5/5 [=========================

In [30]:
model.save('best_model.h5')

In [34]:
model = keras.models.load_model('./best_model.h5')

In [35]:
model.evaluate(x_test, y_test)

4/4 [==============================] - 0s 63ms/step - loss: 0.4127 - accuracy: 0.7879


[0.4126650393009186, 0.7878788113594055]

In [36]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.4126650393009186
Test accuracy: 0.7878788113594055


In [37]:
from sklearn.metrics import classification_report, confusion_matrix

# Use the trained CNN model to predict on the test data
y_pred = model.predict(x_test)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Convert the true labels to class labels
y_true_labels = np.argmax(y_test, axis=1)

# Print the classification report and confusion matrix
print(classification_report(y_true_labels, y_pred_labels))
print(confusion_matrix(y_true_labels, y_pred_labels))

4/4 [==============================] - 0s 52ms/step
              precision    recall  f1-score   support

           0       0.75      0.88      0.81        50
           1       0.85      0.69      0.76        49

    accuracy                           0.79        99
   macro avg       0.80      0.79      0.79        99
weighted avg       0.80      0.79      0.79        99

[[44  6]
 [15 34]]


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"